# Teste técnico QUOD - Limpeza e Análise de Dados de Vendas

In [3]:
import pandas as pd
import numpy as np
import random

### Criação do nosso Dataset

 Definição de produtos e categorias

In [11]:
produtos_categorias = {
    "Notebook": "Eletrônicos",
    "Smartphone": "Eletrônicos",
    "Fone de Ouvido": "Eletrônicos",
    "Camiseta": "Vestuário",
    "Calça Jeans": "Vestuário",
    "Tênis": "Vestuário",
    "Cadeira Gamer": "Móveis",
    "Mesa de Escritório": "Móveis",
    "Sofá": "Móveis",
    "Geladeira": "Eletrodomésticos",
    "Micro-ondas": "Eletrodomésticos",
    "Liquidificador": "Eletrodomésticos"
}

Gerar preço de acordo com a categoria

In [12]:
def gerar_preco(categoria):
    if categoria == "Eletrônicos":
        return round(random.uniform(200, 6000), 2)
    elif categoria == "Vestuário":
        return round(random.uniform(50, 500), 2)
    elif categoria == "Móveis":
        return round(random.uniform(300, 3000), 2)
    elif categoria == "Eletrodomésticos":
        return round(random.uniform(150, 4000), 2)


Gerando o dataset

In [13]:
np.random.seed(42)
random.seed(42)

num_registros = 50
datas = pd.date_range("2023-01-01", "2023-12-31").to_list()

dados = []
for i in range(1, num_registros + 1):
    produto = random.choice(list(produtos_categorias.keys()))
    categoria = produtos_categorias[produto]
    quantidade = random.randint(1, 10)
    preco = gerar_preco(categoria)
    data = random.choice(datas)
    dados.append([i, data.strftime("%d/%m/%Y"), produto, categoria, quantidade, preco])


Criação do dataframe (df)

In [14]:
df = pd.DataFrame(dados, columns=["ID", "Data", "Produto", "Categoria", "Quantidade", "Preço"])

print(df.head())

   ID        Data      Produto         Categoria  Quantidade   Preço
0   1  21/05/2023  Micro-ondas  Eletrodomésticos           2  246.29
1   2  22/02/2023     Camiseta         Vestuário           4  112.79
2   3  05/08/2023  Micro-ondas  Eletrodomésticos           9  484.71
3   4  30/04/2023     Notebook       Eletrônicos           1  743.43
4   5  12/04/2023         Sofá            Móveis          10  371.65


Criação do documento em CSV do nosso dataframe

In [15]:
df.to_csv("vendas_simuladas.csv", index=False, encoding="utf-8-sig")


### Data Cleaning
A partir daqui será feito a limpeza do nosso dataframe, para simular um dataset real, foram feitas mudanças de forma manual, incluindo mais linhas duplicadas e removendo alguns valores, simulando um cenário real

In [21]:
df = pd.read_csv("vendas_simuladas.csv")

In [22]:
print("\nTipos de dados:\n", df.dtypes)


Tipos de dados:
 ID              int64
Data           object
Produto        object
Categoria      object
Quantidade      int64
Preço         float64
dtype: object


In [23]:
df.head()

,ID,Data,Produto,Categoria,Quantidade,Preço
0,1,21/05/2023,Micro-ondas,Eletrodomésticos,2,246.29
1,2,22/02/2023,Camiseta,Vestuário,4,112.79
2,3,05/08/2023,Micro-ondas,Eletrodomésticos,9,484.71
3,4,30/04/2023,Notebook,Eletrônicos,1,743.43
4,5,12/04/2023,Sofá,Móveis,10,371.65


1° Passo:  
Verificação de valores faltantes:
Checa se há células vazias no dataset com df.isnull().sum().

In [24]:

print("Valores faltantes antes do tratamento:\n", df.isnull().sum())


Valores faltantes antes do tratamento:
 ID            0
Data          0
Produto       1
Categoria     1
Quantidade    0
Preço         0
dtype: int64


2° Passo:  

Tratamento de valores faltantes:
Substitui dados ausentes —

Aqui foi usado estratégias diferentes por tipo de coluna

“Produto” e “Categoria” recebem rótulos genéricos;

“Quantidade” recebe a mediana;

“Preço” recebe a média.



In [25]:
df["Produto"] = df["Produto"].fillna("Produto Desconhecido")
df["Categoria"] = df["Categoria"].fillna("Outros")
df["Quantidade"] = df["Quantidade"].fillna(df["Quantidade"].median())
df["Preço"] = df["Preço"].fillna(df["Preço"].mean())


3° Passo 

Remoção de duplicatas:

In [26]:
df = df.drop_duplicates()

4° Passo

Conversão de tipos de dados:
Garante que cada coluna tenha o tipo correto:

ID e Quantidade → inteiros;

Preço → decimal (float);

Data → formato de data (datetime).

In [27]:
df["ID"] = df["ID"].astype(int)
df["Quantidade"] = df["Quantidade"].astype(int)
df["Preço"] = df["Preço"].astype(float)
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y", errors="coerce")

Print dos resultados


In [30]:
print("\nTipos de dados após limpeza:\n", df.dtypes)
print("\nPrimeiras linhas após limpeza:\n", df.head())


Tipos de dados após limpeza:
 ID                     int64
Data          datetime64[ns]
Produto               object
Categoria             object
Quantidade             int64
Preço                float64
dtype: object

Primeiras linhas após limpeza:
    ID       Data      Produto         Categoria  Quantidade   Preço
0   1 2023-05-21  Micro-ondas  Eletrodomésticos           2  246.29
1   2 2023-02-22     Camiseta         Vestuário           4  112.79
2   3 2023-08-05  Micro-ondas  Eletrodomésticos           9  484.71
3   4 2023-04-30     Notebook       Eletrônicos           1  743.43
4   5 2023-04-12         Sofá            Móveis          10  371.65


Verificando novamente se houve realmente as mudanças

In [29]:
dup = df[df.duplicated()].shape[0]
print(f"Há {dup} entradas duplicadas no {df.shape[0]} nesse dataset.")

print("Valores faltantes antes do tratamento:\n", df.isnull().sum())

Há 0 entradas duplicadas no 48 nesse dataset.
Valores faltantes antes do tratamento:
 ID            0
Data          0
Produto       0
Categoria     0
Quantidade    0
Preço         0
dtype: int64


Salvando o dataset limpo em um arquivo data_clean.csv.

In [17]:
df.to_csv("data_clean.csv", index=False, encoding="utf-8-sig")

Utilizando o dataset limpo, foi realizado as seguintes análises:

Calcular o total de vendas (Quantidade * Preço) por produto.


In [18]:
# Criando a coluna com o total de vendas (Quantidade * Preço)
df["Total_Venda"] = df["Quantidade"] * df["Preço"]

# Calculando o total de vendas por produto
vendas_por_produto = df.groupby("Produto")["Total_Venda"].sum().reset_index()


Identificando o produto com o maior número de vendas totais.

In [19]:
produto_top = vendas_por_produto.loc[vendas_por_produto["Total_Venda"].idxmax()]

Resultado:

In [20]:
print("Total de Vendas por Produto")
print(vendas_por_produto.sort_values(by="Total_Venda", ascending=False))

print("\nProduto com Maior Total de Vendas ")
print(produto_top)

Total de Vendas por Produto
               Produto  Total_Venda
7          Micro-ondas     90570.02
9           Smartphone     55699.79
4            Geladeira     48943.11
3       Fone de Ouvido     44526.22
0        Cadeira Gamer     37610.83
8             Notebook     28917.11
5       Liquidificador     21698.01
10                Sofá     16680.41
2             Camiseta      7078.93
1          Calça Jeans      2623.33
6   Mesa de Escritório      1938.70
11               Tênis       470.08

Produto com Maior Total de Vendas 
Produto        Micro-ondas
Total_Venda       90570.02
Name: 7, dtype: object
